# Лабораторная работа №1. <br> Метрические алгоритмы классификации

Выполнила: Кузнецова Екатерина

## 1. Постановка задачи

1.	На языке Python программно реализовать два метрических алгоритма классификации: Naive Bayes и K Nearest Neighbours
2.	Сравнить работу реализованных алгоритмов с библиотечными из scikit-learn
3.	Для тренировки, теста и валидации использовать один из предложенных датасетов (либо найти самостоятельно и внести в таблицу)
4.	Сформировать краткий отчет (постановка задачи, реализация, эксперимент с данными, полученные характеристики, вывод

## 2. Исходные данные

Датасет: http://archive.ics.uci.edu/ml/datasets/Zoo<br>
Предметная область: обитатели зоопарка


Количество записей: 101<br>
Количество атрибутов: 17


Атрибуты:

1. Название животного (строка, уникальный для каждого экземпляра)
2. Наличие волос (логический тип)
3. Наличие перьев (логический тип)
4. Яйца (логический тип)
5. Млекопитающий (логический тип)
6. Умеет летать (логический тип)
7. Водный (логический тип)
8. Хищник (логический тип)
9. Наличие зубов (логический тип) 
10. Наличие позвоночника (логический тип)
11. Дышит воздухом (логический тип)
12. Ядовитость (логический тип)
13. Наличие плавников (логический тип)
14. Количество ног (набор целочисленных значений: {0,2,4,5,6,8})
15. Наличие хвоста (логический тип)
16. Является домашним (логический тип)
17. Catsize (логический тип)
18. Тип (целочисленные значения в диапазоне [1,7])

## 3. Ход работы

1. Реализация алгоритма Naive Bayes.

In [3]:
import math
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.naive_bayes import GaussianNB

# загрузка датасета
def load_data(filename):
    return pd.read_csv(filename, header=None).values

# разделение датасета на тестовую и обучающую выборку
def split_dataset(test_size):
    dataset = load_data('zoo.data.csv')
    animal_attr = dataset[:, 1:-1]
    animal_class = dataset[:, -1]
    animal_class = animal_class.astype(np.int64, copy=False)
    data_train, data_test, class_train, class_test = train_test_split(animal_attr, animal_class,
 test_size=test_size, random_state=55)
    return data_train, class_train, data_test, class_test


# Разделяет обучающую выборку по классам таким образом, чтобы можно было получить все элементы, принадлежащие
# определенному классу.
def separate_by_class(data_train, class_train):
    classes_dict = {}
    for i in range(len(data_train)):
        classes_dict.setdefault(class_train[i], []).append(data_train[i])
    return classes_dict


# инструменты для обобщения данных
def mean(numbers):  # Среднее значение
    return sum(numbers) / float(len(numbers))


def stand_dev(numbers):  # вычисление дисперсии
    var = sum([pow(x - mean(numbers), 2) for x in numbers]) / float(len(numbers) - 1)
    return math.sqrt(var)


def summarize(data_train):  # обобщение данных
    # Среднее значение и среднеквадратичное отклонение для каждого атрибута
    summaries = [(mean(att_numbers), stand_dev(att_numbers)) for att_numbers in zip(*data_train)]
    return summaries


# Обучение классификатора
def summarize_by_class(data_train, class_train):
    # Разделяет обучающую выборку по классам таким образом, чтобы можно было получить все элементы, принадлежащие определенному классу.
    classes_dict = separate_by_class(data_train, class_train)
    summaries = {}
    for class_name, instances in classes_dict.items():
        # Среднее значение и среднеквадратичное отклонение атрибутов для каждого класса входных данных
        summaries[class_name] = summarize(instances)
    return summaries


# вычисление апостериорной вероятности принадлежности объекта к определенному классу
def calc_probability(x, mean, stdev):
    if stdev == 0:
        stdev += 0.000001  # добавляем эпсилон, если дисперсия равна 0
    exponent = math.exp(-(math.pow(x - mean, 2) / (2 * math.pow(stdev, 2))))
    return (1 / (math.sqrt(2 * math.pi) * stdev)) * exponent


# вычисление вероятности принадлежности объекта к каждому из классов
def calc_class_probabilities(summaries, instance_attr):
    probabilities = {}
    for class_name, class_summaries in summaries.items():
        probabilities[class_name] = 1.0
        for i in range(len(class_summaries)):
            mean, stdev = class_summaries[i]
            x = float(instance_attr[i])
            probabilities[class_name] *= calc_probability(x, mean, stdev)
    return probabilities


# классификация одного объекта
def predict_one(summaries, instance_attr):
    # вычисление вероятности принадлежности объекта к каждому из классов
    probabilities = calc_class_probabilities(summaries, instance_attr)
    best_class, max_prob = None, -1
    for class_name, probability in probabilities.items():
        if best_class is None or probability > max_prob:
            max_prob = probability
            best_class = class_name
    return best_class


# классификация тестовой выборки
def predict(summaries, data_test):
    predictions = []
    for i in range(len(data_test)):
        result = predict_one(summaries, data_test[i])
        predictions.append(result)
    return predictions


# сравнение результатов классификации с реальными, вычисление точности классификации
def calc_accuracy(summaries, data_test, class_test):
    correct_answ = 0
    # классификация тестовой выборки
    predictions = predict(summaries, data_test)
    for i in range(len(data_test)):
        if class_test[i] == predictions[i]:
            correct_answ += 1
    return correct_answ / float(len(data_test))

Сравнение работы реализованного алгоритма с библиотечным:

In [4]:
def main():
    data_train, class_train, data_test, class_test = split_dataset(0.3)
    summaries = summarize_by_class(data_train, class_train)
    accuracy = calc_accuracy(summaries, data_test, class_test)
    print('myNBClass ', 'Accuracy: ', accuracy)

    clf = GaussianNB()
    clf.fit(data_train, class_train)
    print('sklNBClass ', 'Accuracy: ', clf.score(data_test, class_test))


main()

myNBClass  Accuracy:  0.9354838709677419
sklNBClass  Accuracy:  0.967741935484


2. Реализация алгоритма K Nearest Neighbours

In [5]:
from __future__ import division

import pandas as pd
import numpy as np
import operator
from sklearn.model_selection import train_test_split
from math import sqrt
from collections import Counter

from sklearn.neighbors import KNeighborsClassifier


def load_data():
    dataset = pd.read_csv('zoo.data.csv', header=None).values
    animal_attr = dataset[:, 1:-1]
    animal_class = dataset[:, -1]
    animal_class = animal_class.astype(np.int64, copy=False)
    return train_test_split(animal_attr, animal_class, test_size=0.35)


# евклидово расстояние от объекта №1 до объекта №2
def euclidean_distance(instance1, instance2):
    squares = [(i - j) ** 2 for i, j in zip(instance1, instance2)]
    return sqrt(sum(squares))


# рассчет расстояний до всех объектов в датасете
def get_neighbours(instance, data_train, class_train, k):
    distances = []
    for i in data_train:
        distances.append(euclidean_distance(instance, i))
    distances = tuple(zip(distances, class_train))
    # cортировка расстояний по возрастанию
    # k ближайших соседей
    return sorted(distances, key=operator.itemgetter(0))[:k]


# определение самого распространенного класса среди соседей
def get_response(neigbours):
    return Counter(neigbours).most_common()[0][0][1]


# классификация тестовой выборки
def get_predictions(data_train, class_train, data_test, k):
    predictions = []
    for i in data_test:
        neigbours = get_neighbours(i, data_train, class_train, k)
        response = get_response(neigbours)
        predictions.append(response)
    return predictions


# измерение точности
def get_accuracy(data_train, class_train, data_test, class_test, k):
    predictions = get_predictions(data_train, class_train, data_test, k)
    mean = [i == j for i, j in zip(class_test, predictions)]
    return sum(mean) / len(mean)

Сравнение работы реализованного алгоритма с библиотечным:

In [6]:
def main():
    data_train, data_test, class_train, class_test = load_data()
    print('myKNClass', 'Accuracy: ', get_accuracy(data_train, class_train, data_test, class_test, 15))

    clf = KNeighborsClassifier(n_neighbors=15)
    clf.fit(data_train, class_train)
    print('sklKNClas', 'Accuracy: ', clf.score(data_test, class_test))


main()

myKNClass Accuracy:  0.75
sklKNClas Accuracy:  0.75


## 4. Заключение

В ходе лабораторной работы на языке Python были программно реализованы два метрических алгоритма классификации: Naive Bayes и K Nearest Neighbours. Для оценки точности классификации и сравнения полученных алгоритмов с аналогичными, реализованными в библиотеке scikit-learn, они были применены к датасету "Зоопарк".<br> Результаты тестирования показали, что библиотечная реализация Наивного Байесовского классификатора работает немного лучше, нежели воплощенная в данной лабораторной работе, но обе они показывают неплохие результаты классификации (более 93% правильно классифицированных объектов).<br> Обе реализации метода K ближайших соседей работают с одинаковой точностью (75%), но значитально проигрывают по качеству классификации Байесовскому алгоритму.